In [1]:
import sys

In [2]:
sys.path.insert(0, '../codes/new_transformers_branch/transformers/src')

In [3]:
import torch as t
from glob import glob
import pandas as pd
import dill as pickle
import re
import numpy as np
from tqdm import tqdm
import spacy
from spacy import displacy
from transformers import DebertaV2TokenizerFast
import h5py

In [4]:
# 토크나이저
tokenizer = DebertaV2TokenizerFast.from_pretrained('microsoft/deberta-v3-large')
tokenizer.model_max_length = 2048

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# train_data
data = pd.read_csv('../../../feedback-prize-2021/train.csv')
# florida -> Location_Name으로 변경 (xx학교 -> School name 이렇게 사용하는듯)
data.loc[data.discourse_id==1623258656795.0, 'discourse_text'] =  data.loc[data.discourse_id==1623258656795.0, 'discourse_text'].map(lambda x: x.replace('florida', 'LOCATION_NAME')).values

In [6]:
# 라벨
label_names = ['None', 'Lead', 'Position', 'Evidence', 'Claim',
               'Concluding Statement', 'Counterclaim', 'Rebuttal']

In [7]:
colors = {
            'Lead': '#8000ff',
            'Position': '#2b7ff6',
            'Evidence': '#2adddd',
            'Claim': '#80ffb4',
            'Concluding Statement': 'd4dd80',
            'Counterclaim': '#ff8042',
            'Rebuttal': '#ff0000'
         }
         
options = {"ents": list(colors.keys()), "colors": colors}
token_maps = dict(zip(colors, range(1, 2 * len(colors), 2)))
token_maps

{'Lead': 1,
 'Position': 3,
 'Evidence': 5,
 'Claim': 7,
 'Concluding Statement': 9,
 'Counterclaim': 11,
 'Rebuttal': 13}

In [8]:
regexp = re.compile('[0-9a-zA-z]')

In [9]:
# ? 이해 필요
def make_more_targets(targets):
    # target -> NER target
    linkage = np.zeros((len(targets), 2), 'f4')
    class_index = np.zeros((len(targets),), 'f4')
    linkage_mask = np.ones((len(targets),), 'f4')
    
    current_target = -2
    for ix in range(1, len(targets) -1):
        if ((current_target % 2 == 0 and current_target == targets[ix]) or (targets[ix] == current_target + 1 and current_target %2 == 1)):
        # current_target이 처음나오는 start이고  current target이 ix번째 target과 같을때 or current_target이 나온적이 있는 target이고  ix번째 target과 current_target + 1이 같을때
        # linkage [0] -> 앞에것과 같은가?, linkage[1] -> 뒤에것과 같은가?
            linkage[ix - 1, 1] = 1
            linkage[ix, 0] = 1
            
        current_target = targets[ix]
        # class_index -> NER 테스크의 시작점 구분없이
        class_index[:] = [x // 2 for x in targets + 1]

    #link_sums : 3-> 앞뒤가 같은것, 2 -> 앞에만 같은것(마지막이라는 의미), 1 -> 뒤에만 같은것(처음이라는 의미)
    link_sums = (linkage * np.array([2, 1])).sum(-1).astype('i4')
    
    # bi [1, 0] -> 앞이 다른경우 (처음 나온경우) [0, 1] -> 앞이 같은경우 (나온적이 있는경우) ->binary
    bi =  np.zeros((len(targets), 2), 'f4')
    bi[link_sums < 2, 0] = 1
    bi[link_sums >= 2, 1] = 1
    #bio -> target이 0 인경우 [0, 0] ex) padding
    bio = np.array(bi)
    bio[targets==0] = 0
    #bies -> link_sum : 0 -> (0, 0, 0, 1),  link_sum : 1 -> (1, 0, 0, 0),  link_sum : 2 -> (0, 0, 1, 0),  link_sum : 3 -> (0, 1, 0, 0), 
    bies =  np.zeros((len(targets), 4), 'f4')
    bies[:, :2] = bi
    bies[link_sums == 0] = (0, 0, 0, 1)
    bies[link_sums == 2] = (0, 0, 1, 0)
    #bieso -> target이 0 인경우 [0, 0] ex) padding
    bieso = np.array(bies)
    bieso[targets==0] = 0
    return class_index, bi, bio, bies, bieso

In [10]:
def combine_labels(class_index, bi, bio, bies, bieso):
    # class_index와 bi, bio, bies, bideso 결합
    combined_bi = class_index * 2 + bi[:, 0]
    combined_bies = class_index * 4 + bies @ np.array([0, 1, 2, 3])
    
    # class_index 라벨이 0(padding)이 아닌곳 
    non_o_index = np.where(class_index != 0)[0]
    combined_bieso = np.array(class_index)
    combined_bieso[non_o_index] = (class_index[non_o_index] - 1) * 4 + bieso[non_o_index] @ np.array([1, 2, 3, 4])
    combined_bio = np.array(class_index)
    combined_bio[non_o_index] = (class_index[non_o_index] - 1) * 2 + bio[non_o_index] @ np.array([1, 2])
    return combined_bi, combined_bio, combined_bies, combined_bieso

In [11]:
import h5py

In [12]:
# data_file 생성
data_file = h5py.File('../data_file/deberta_spm_data_v2.h5py', 'w')

In [13]:
# txt 개수 + 1 -> 1을 왜 더함?
num_texts = len(glob('../../../feedback-prize-2021/train/*.txt')) + 1

In [14]:
# (numtext, 2048) zeros
tokens_dataset = data_file.create_dataset('tokens', (num_texts, 2048), 'i8')
attention_masks_dataset = data_file.create_dataset('attention_masks', (num_texts, 2048), 'f4')
token_offsets_dataset = data_file.create_dataset('token_offsets', (num_texts, 2048, 2), 'i4')
class_labels_dataset = data_file.create_dataset('class_labels', (num_texts, 2048, 8), 'f4')
num_tokens_dataset = data_file.create_dataset('num_tokens', (num_texts, 2048), 'i4')

bi_labels_dataset = data_file.create_dataset('bi_labels', (num_texts, 2048, 2), 'f4')
bio_labels_dataset = data_file.create_dataset('bio_labels', (num_texts, 2048, 2), 'f4')
bies_labels_dataset = data_file.create_dataset('bies_labels', (num_texts, 2048, 4), 'f4')
bieso_labels_dataset = data_file.create_dataset('bieso_labels', (num_texts, 2048, 4), 'f4')

cbi_labels_dataset = data_file.create_dataset('cbi_labels', (num_texts, 2048, 16), 'f4')
cbio_labels_dataset = data_file.create_dataset('cbio_labels', (num_texts, 2048, 15), 'f4')
cbies_labels_dataset = data_file.create_dataset('cbies_labels', (num_texts, 2048, 32), 'f4')
cbieso_labels_dataset = data_file.create_dataset('cbieso_labels', (num_texts, 2048, 29), 'f4')

In [15]:
for dataset in tqdm((tokens_dataset, 
                     attention_masks_dataset,
                     token_offsets_dataset,
                     class_labels_dataset,
                     num_tokens_dataset,
                     bi_labels_dataset,
                     bio_labels_dataset,
                     bies_labels_dataset,
                     bieso_labels_dataset,
                     cbi_labels_dataset,
                     cbio_labels_dataset,
                     cbies_labels_dataset,
                     cbieso_labels_dataset,)):
    dataset[-1] = 0
    


100% 13/13 [00:00<00:00, 51.25it/s]


In [16]:
# onehot encoding으로 변경
def make_one_hot(indices, num_labels):
    array = np.zeros((len(indices), num_labels))
    array[np.arange(len(indices)), indices.astype('i4')] = 1
    return array

In [17]:
id_to_ix_map = {filename: ix for ix, filename in enumerate(glob('../../../feedback-prize-2021/train/*.txt'))}
with open('../data_file/id_to_ix_map.pickle', 'wb') as f:
    import dill as pickle
    pickle.dump(id_to_ix_map, f)

In [18]:
# '/n' -> ‽로 변경
fix_text = lambda x: x.replace('\n', '‽')

In [19]:
from collections import Counter

In [20]:
# 에러
broken_indices = []

In [21]:
print(tokenizer.decode([126861]))
print(tokenizer.decode([128000]))

‽
[MASK]


In [ ]:
id_to_ix_map = {}
for filename_ix, filename in tqdm(enumerate(glob('../../../feedback-prize-2021/train/*.txt')), total = num_texts-1):
    ID = filename.split('/')[-1].split('.')[0]
    with open(filename) as f:
        # /n 제거
        text = fix_text(f.read().strip())

    tokenizer_outs = tokenizer(text, return_offsets_mapping=True)
    # 토크나이저 ‽일경우 -> MASK 토큰으로 변경
    tokenizer_outs['input_ids'] = [x if x != 126861 else 128000 for x in tokenizer_outs['input_ids']]
    # data -> train.csv, txt파일과 같은 ID를가지는 row들 중에서 discourse_start순으로 정렬
    text_data = data.loc[data.id==ID].sort_values('discourse_start')

    ent_boundaries = []
    # pointer -> text에서 시작하는 시점
    pointer = 0
    for row_id, row in text_data.iterrows():
        # discourse_text에서 '/n' -> ‽로 변경
        entity_text = fix_text(row.discourse_text.strip())
        # 정규표현식
        entity_text = entity_text[next(regexp.finditer(entity_text)).start():]
        # print(entity_text.split()[0])

        if len(entity_text.split()[0]) == 1 and pointer != 0:
            # i, a, 등등 처음 오는 단어가 한자리고 pointer가 0이 아닐때 -> 어떤의미?
            # entity_start_ix -> pointer에서부터 entity_text가 나올때 까지의 index
            entity_start_ix = text[pointer:].index(entity_text)

            # prev_text -> entity_text가 나오기 이전까지의 text
            prev_text = text[:pointer + entity_start_ix]

            # 시작하는 시점이 0보다 prev_text의 마지막이 알파벳일경우 
            if pointer + entity_start_ix > 0 and prev_text[-1].isalpha():
                broken_indices.append((filename_ix, ID))
                print('cut entity ', filename_ix, ID)
                cut_word_chunk_size = len(prev_text.split()[-1])
                if cut_word_chunk_size > 1:
                    entity_text = entity_text[next(regexp.finditer(entity_text[1:])).start() + 1:]
        
        if row.discourse_id in (1620147556527.0, 1622983056026.0):
            # id가 1620147556527, 1622983056026일경우 pointer + 10 -> 이유는?
            pointer += 10

        offset = text[pointer:].index(entity_text)
        # text에서 시작하는 시점
        starts_at = offset + pointer
        # (text에서 시작하는 시점, 끝나는 시점, row.discourse_type)
        ent_boundaries.append((starts_at, starts_at + len(entity_text), row.discourse_type))
        # 다음 pointer는 starts_at + len(entity_text) -> 부터 시작 -> 시간감소를 위한것?
        pointer = starts_at + len(entity_text)
    
    # (location, ent, start or end)
    all_boundaries = [(z, x[-1], t) for x in ent_boundaries for z, t in zip(x[:2], ('start', 'end'))]
    current_target = 0
    targets = np.zeros(len(tokenizer_outs['input_ids']), 'i8')
    token_positions = np.vstack(tokenizer_outs['offset_mapping']).astype('i4')

    for token_ix in range(len(tokenizer_outs['input_ids'])):
        # 단어 시작 끝?
        token_start_ix, token_end_ix = tokenizer_outs['offset_mapping'][token_ix]
        # all_boundaries 가장 앞에있는것이 end 일경우 -> token_end_ix 가 all_boundaries[0] (location) 보다 크거나 같을
        # all_boundaries 가장 앞에있는것이 start 일경우 -> token_end_ix 가 all_boundaries[0] (location) 보다 클경우
        if token_end_ix != 0 and (all_boundaries[0][2] == 'end' and token_end_ix >= all_boundaries[0][0])\
                            or (all_boundaries[0][2] == 'start' and token_end_ix > all_boundaries[0][0]):

            if all_boundaries[0][2] == 'start':
                #target
                current_target = token_maps[all_boundaries[0][1]]
                targets[token_ix] = current_target
                # offset mapping의 end가 start 위치와 같을경우 0 -> "" 빈칸이기 때문??
                if token_end_ix == all_boundaries[1][0]:
                    current_target = 0
                    # 처음이 끝이기 때문에 pop
                    all_boundaries.pop(0)
                else:
                    # 처음 나온 start 다음은 + 1
                    current_target += 1
            else:
                # end 일경우 token_end_ix가 다음 all_boundaries location보다 클 때
                if len(all_boundaries) > 1 and token_end_ix > all_boundaries[1][0]:
                    if token_start_ix >= all_boundaries[1][0]:
                        assert text[all_boundaries[0][0] - 1] == '¨'
                    all_boundaries.pop(0)
                    current_target = token_maps[all_boundaries[0][1]]
                    targets[token_ix] = current_target
                    current_target += 1

                else:
                    if token_start_ix >= all_boundaries[0][0]:
                        current_target = 0
                    targets[token_ix] = current_target
                    current_target = 0
            all_boundaries.pop(0)
            if not all_boundaries:
                break
        else:
            # 변화가 없는 위치 -> current_target
            targets[token_ix] = current_target
            
    # target -> NER targets
    class_index, bi, bio, bies, bieso = make_more_targets(targets)
    combined_bi, combined_bio, combined_bies, combined_bieso = combine_labels(class_index, bi, bio, bies, bieso)
    assert (combined_bio[1:-1] == targets[1:-1]).all()
    num_tokens = len(targets)
    
    # 남은것들은 padding -> 0
    tokens_dataset[filename_ix, :num_tokens] = tokenizer_outs['input_ids']
    tokens_dataset[filename_ix, num_tokens:] = 0
    attention_masks_dataset[filename_ix, :num_tokens] = tokenizer_outs['attention_mask']
    attention_masks_dataset[filename_ix, num_tokens:] = 0
    token_offsets_dataset[filename_ix, :num_tokens] = token_positions
    token_offsets_dataset[filename_ix, num_tokens:] = 0
    class_labels_dataset[filename_ix, :num_tokens] = make_one_hot(class_index, 8)
    class_labels_dataset[filename_ix, num_tokens:] = 0
    num_tokens_dataset[filename_ix] = num_tokens
    bi_labels_dataset[filename_ix, :num_tokens] = bi
    bi_labels_dataset[filename_ix, num_tokens:] = 0
    bio_labels_dataset[filename_ix, :num_tokens] = bio
    bio_labels_dataset[filename_ix, num_tokens:] = 0
    bies_labels_dataset[filename_ix, :num_tokens] = bies
    bies_labels_dataset[filename_ix, num_tokens:] = 0
    bieso_labels_dataset[filename_ix, :num_tokens] = bieso
    bieso_labels_dataset[filename_ix, num_tokens:] = 0
    cbi_labels_dataset[filename_ix, :num_tokens] = make_one_hot(combined_bi, 16)
    cbi_labels_dataset[filename_ix, num_tokens:] = 0
    cbio_labels_dataset[filename_ix, :num_tokens] = make_one_hot(combined_bio, 15)
    cbio_labels_dataset[filename_ix, num_tokens:] = 0
    cbies_labels_dataset[filename_ix, :num_tokens] = make_one_hot(combined_bies, 32)
    cbies_labels_dataset[filename_ix, num_tokens:] = 0
    cbieso_labels_dataset[filename_ix, :num_tokens] = make_one_hot(combined_bieso, 29)
    cbieso_labels_dataset[filename_ix, num_tokens:] = 0

  0% 7/15594 [00:01<1:06:56,  3.88it/s]

In [ ]:
data_file.close()